<a href="https://colab.research.google.com/github/hBar2013/DS-Unit-2-Classification-1/blob/master/module3-random-forests/kim_lowry_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install category_encoders

     |████████████████████████████████| 92kB 2.4MB/s 


In [0]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv


In [0]:
import category_encoders as ce
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline



train_features_file = 'train_features.csv'
train_labels_file = 'train_labels.csv'
test_features_file = 'test_features.csv'

train_feat = pd.read_csv(train_features_file)
train_labels = pd.read_csv(train_labels_file)
test_feat = pd.read_csv(test_features_file)

In [43]:
X_train = train_feat
y_train = train_labels['status_group']

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size=0.80, test_size=0.20, 
    stratify=y_train, random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((47520, 40), (11880, 40), (47520,), (11880,))

In [23]:
X_train.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,47520.000000,47520.000000,47520.000000,47520.000000,4.752000e+04,47520.000000,47520.000000,47520.000000,47520.000000,47520.000000
mean,37037.915699,321.925261,669.567656,34.082431,-5.705946e+00,0.477736,15.258291,5.616751,179.805513,1302.768939
std,21412.099719,3197.240487,693.005745,6.552053,2.941332e+00,13.312977,17.530228,9.621230,463.081564,950.955437
min,0.000000,0.000000,-63.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18482.750000,0.000000,0.000000,33.092142,-8.528215e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,36986.500000,0.000000,372.500000,34.910740,-5.021436e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55450.250000,25.000000,1320.000000,37.170578,-3.327185e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.344301,-2.000000e-08,1776.000000,99.000000,80.000000,15300.000000,2013.000000


In [10]:
X_train.query('gps_height == 0').gps_height.count()

16305

In [9]:
X_train.query('longitude == 0').longitude.count()

1442

In [0]:
#function to engineer some other features

def clean_data(X):
    X = X.copy()
    X['latitude'] = X['latitude'].replace(-2e-08, np.nan) #clean up latitude
    
     
    # Setting 0 to NaN in numeric fields of interest except for gps_height. Handle
    # gps_height first: only set to NaN if longitude is 0
    X.loc[X['longitude'] == 0, 'gps_height'] = np.nan
    
    #now set 0 to NaN for other columns of interest
    columns = ['construction_year', 'population', 'latitude']
    for col in columns:
       X[col] = X[col].replace(0, np.nan)
      
    return X

In [0]:
X_train = clean_data(X_train)
X_val = clean_data(X_val)
test_feat = clean_data(test_feat)

In [0]:
# list of features for this round
features = ['construction_year', 'population', 'latitude', 'longitude',
            'gps_height', 'region_code', 'district_code', 'payment_type',
            'waterpoint_type', 'quantity', 'extraction_type', 'permit', 
            'management'] 

In [0]:
X_train_subset = X_train[features]
X_val_subset = X_val[features]
test_subset = test_feat[features]

In [48]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='median'), 
    #SVC(gamma='scale', kernel='rbf', probability=True)
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

# Fit on train, score on val
pipeline.fit(X_train_subset, y_train)
print('Validation Accuracy', pipeline.score(X_val_subset, y_val))

Validation Accuracy 0.5582491582491582


In [0]:
y_pred = pipeline.predict(test_subset)

In [0]:
sumbission_sample_file = 'sample_submission.csv'
sample_submission = pd.read_csv(sumbission_sample_file)

In [0]:
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('kel-submission-05.csv', index=False)

In [32]:
!head kel-submission-05.csv

id,status_group
50785,functional
51630,functional
17168,functional
45559,non functional
49871,functional
52449,functional
24806,functional
28965,non functional
36301,non functional


In [0]:
from google.colab import files
files.download('kel-submission-05.csv')